In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv(r'E:\Santander Recommendation\train_ver2.csv')

C:\Users\liamc\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = train.loc[train['fecha_dato']=='2016-05-28']

In [4]:
product_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1',
                'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1',
                'ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_dela_fin_ult1','ind_deme_fin_ult1',
                'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_nom_pens_ult1',
                'ind_nomina_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1',
                'ind_recibo_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1']

In [5]:
from lightfm.data import Dataset

C:\Users\liamc\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [6]:
dataset = Dataset()
dataset.fit(train['ncodpers'],product_cols)

In [7]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 931453, num_items 24.


In [8]:
labels = train[product_cols]

In [9]:
labels['ncodpers'] = train.ncodpers

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
labels = labels.set_index("ncodpers")

In [11]:
stacked_labels = labels.stack()

In [12]:
filtered_labels = stacked_labels.reset_index()

In [13]:
filtered_labels.columns = ["ncodpers", "product", "interaction"]

In [14]:
interactionsdf = filtered_labels.loc[filtered_labels['interaction']==1]

In [15]:
(interactions, weights) = dataset.build_interactions((interactionsdf.iloc[x,0], interactionsdf.iloc[x,1])
                                                     for x in range(len(interactionsdf)))

In [16]:
len(interactionsdf)

1240538

In [17]:
print(repr(interactions))

<931453x24 sparse matrix of type '<class 'numpy.int32'>'
	with 1240538 stored elements in COOrdinate format>


In [18]:
#do some data cleaning before adding in user features

dirtyrows = train[train.ind_empleado.isnull()].index

train = train.drop(dirtyrows, axis = 0)

train.ind_empleado = train.ind_empleado.fillna('N')

train.canal_entrada = train.canal_entrada.fillna("Unknown")

train = train.drop("cod_prov", 1)

train.conyuemp = train.conyuemp.fillna("Unknown")

train.loc[train.indrel_1mes == '1', 'indrel_1mes'] = 'Primary'
train.loc[train.indrel_1mes == '1.0', 'indrel_1mes'] = 'Primary'
train.loc[train.indrel_1mes == 1, 'indrel_1mes'] = 'Primary'
train.loc[train.indrel_1mes == 1.0, 'indrel_1mes'] = 'Primary'
train.loc[train.indrel_1mes == '2', 'indrel_1mes'] = 'Co-owner'
train.loc[train.indrel_1mes == '2.0', 'indrel_1mes'] = 'Co-owner'
train.loc[train.indrel_1mes == 2, 'indrel_1mes'] = 'Co-owner'
train.loc[train.indrel_1mes == 2.0, 'indrel_1mes'] = 'Co-owner'
train.loc[train.indrel_1mes == '3', 'indrel_1mes'] = 'Former Primary'
train.loc[train.indrel_1mes == '3.0', 'indrel_1mes'] = 'Former Primary'
train.loc[train.indrel_1mes == 3, 'indrel_1mes'] = 'Former Primary'
train.loc[train.indrel_1mes == 3.0, 'indrel_1mes'] = 'Former Primary'
train.loc[train.indrel_1mes == '4', 'indrel_1mes'] = 'Former Co-owner'
train.loc[train.indrel_1mes == '4.0', 'indrel_1mes'] = 'Former Co-owner'
train.loc[train.indrel_1mes == 4, 'indrel_1mes'] = 'Former Co-owner'
train.loc[train.indrel_1mes == 4.0, 'indrel_1mes'] = 'Former Co-owner'

train.indrel_1mes = train.indrel_1mes.fillna('Primary')

train.nomprov = train.nomprov.fillna("MADRID")

In [19]:
#rest of feature cleaning
train.renta.fillna(0)
train.renta = train.renta.astype(float)
train.loc[train.renta == 0, 'renta'] = train.renta
train.loc[train.renta.isnull(), "renta"] = train.groupby('nomprov').renta.transform('median')

train.segmento = train[train.renta <= 98000].segmento.fillna("03 - UNIVERSITARIO")
train.segmento = train[train.renta <= 125500].segmento.fillna("02 - PARTICULARES")

train.segmento = train.segmento.fillna("01 - TOP")

train.sexo = train.sexo.fillna("V")

train.tiprel_1mes = train.tiprel_1mes.fillna('I')

train.ind_nomina_ult1 = train.ind_nomina_ult1.fillna(0.0)
train.ind_nom_pens_ult1 = train.ind_nom_pens_ult1.fillna(0.0)

train.loc[train.antiguedad == -999999, 'antiguedad'] = train.antiguedad.median()
train.loc[train.indrel == 99.0, "indrel"] = 0.0

In [20]:
train.drop('ult_fec_cli_1t',axis=1,inplace=True)

In [21]:
feature_cols = ['ind_empleado', 'pais_residencia', 'sexo',
       'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
       'indfall', 'tipodom', 'nomprov', 'ind_actividad_cliente', 'renta',
       'segmento']

In [22]:
features = train[feature_cols]

In [23]:
features['ncodpers'] = train.ncodpers

C:\Users\liamc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [24]:
features = features.set_index("ncodpers")

In [25]:
stacked_features = features.stack()

In [26]:
features = stacked_features.reset_index()

In [27]:
features.columns = ["ncodpers", "feature_name", "feature"]

In [28]:
stacked_features.head()

ncodpers                 
657640    ind_empleado                N
          pais_residencia            ES
          sexo                        H
          age                        68
          fecha_alta         2006-11-17
dtype: object

In [29]:
import json

In [30]:
#features_json = features.to_json()

In [40]:
stacked_json = stacked_features.to_json(orient='table')

In [41]:
pd.__version__

'0.23.4'

In [42]:
type(stacked_json)

str

In [43]:
features_json = json.loads(stacked_json)

In [ ]:
features_json =[]

In [ ]:
features_json['657640']

In [45]:
from itertools import islice

In [46]:
for line in islice(features_json['data'], 20):
    print(json.dumps(line, indent=4))

{
    "ncodpers": 657640,
    "level_1": "ind_empleado",
    "values": "N"
}
{
    "ncodpers": 657640,
    "level_1": "pais_residencia",
    "values": "ES"
}
{
    "ncodpers": 657640,
    "level_1": "sexo",
    "values": "H"
}
{
    "ncodpers": 657640,
    "level_1": "age",
    "values": 68
}
{
    "ncodpers": 657640,
    "level_1": "fecha_alta",
    "values": "2006-11-17"
}
{
    "ncodpers": 657640,
    "level_1": "ind_nuevo",
    "values": 0.0
}
{
    "ncodpers": 657640,
    "level_1": "antiguedad",
    "values": 114
}
{
    "ncodpers": 657640,
    "level_1": "indrel",
    "values": 1.0
}
{
    "ncodpers": 657640,
    "level_1": "indrel_1mes",
    "values": "Primary"
}
{
    "ncodpers": 657640,
    "level_1": "tiprel_1mes",
    "values": "I"
}
{
    "ncodpers": 657640,
    "level_1": "indresi",
    "values": "S"
}
{
    "ncodpers": 657640,
    "level_1": "indext",
    "values": "N"
}
{
    "ncodpers": 657640,
    "level_1": "conyuemp",
    "values": "Unknown"
}
{
    "ncodpers": 6576

In [ ]:
features_json['schema']

In [ ]:
features_json['ncodpers']

In [ ]:
features_json

In [ ]:
features_json['data'][0]['ncodpers']

In [ ]:
features.loc[(657640,'age')]

In [ ]:
user_features = dataset.build_user_features((features_json['data'][x][ncodpers], features_json['data'][x][1])
                                                     for x in range(len(features)))

In [ ]:
from lightfm import LightFM

model = LightFM(no_components=20,learning_schedule='adagrad',loss='warp')

In [ ]:
model.fit(interactions,epochs=40,verbose=True)

In [ ]:
test = pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv')

In [ ]:
users = test.ncodpers.values

In [ ]:
users

In [ ]:
n_users, n_items = dataset.interactions_shape()

In [ ]:
item_ids = np.arange(n_items)
model.predict(0,item_ids)

In [ ]:
dataset.mapping()[1][1170544]

In [ ]:
preds = []
item_ids = np.arange(n_items)
for user_id in users:
    emb_id = dataset.mapping()[1][user_id]
    scores = model.predict(emb_id,item_ids)
    preds.append(scores)
    scores = []

In [ ]:
preds =np.array([np.array(predi) for predi in preds])

In [ ]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)

In [ ]:
preds[0]

In [ ]:
final_month_training_cols = train
final_month_training_cols = final_month_training_cols.drop(product_cols, axis=1)

In [ ]:
#Get the target columns
labels_final_month = train[product_cols]

#Add ncodpers to the dataframe
labels_final_month['ncodpers'] = train.ncodpers

labels_final_month = labels_final_month.set_index("ncodpers")

stacked_labels_final_month = labels_final_month.stack()

filtered_labels_final_month = stacked_labels_final_month.reset_index()

filtered_labels_final_month.columns = ["ncodpers", "product", "newly_added"]

#Only select the rows where there is a new product.
filtered_labels_final_month = filtered_labels_final_month[filtered_labels_final_month["newly_added"] == 1]

#Merge with the training features.
multiclass_final_month = filtered_labels_final_month.merge(final_month_training_cols, on="ncodpers", how="left")

final_month = multiclass_final_month.drop_duplicates(multiclass_final_month, keep='last')

labels_final_month_final = final_month['product']

final_month_ncodpers = final_month.ncodpers
#Remove the columns that are not needed to train the model.
final_month = final_month.drop(['ncodpers','newly_added','product'], axis = 1)

In [ ]:
labels_final_month_final_cat = labels_final_month_final.astype('category').cat.codes

In [ ]:
used_products = pd.DataFrame()
used_products['product'] = labels_final_month_final_cat
used_products['ncodpers'] = final_month_ncodpers
used_products = used_products.drop_duplicates(keep = 'last')

In [ ]:
#create a dictionary to store each product a customer already has
used_recommendation_products = {}
product_cols = np.array(product_cols)
#iterate through used_products and add each one to used_recommendation_products
for idx,row_val in used_products.iterrows():
    used_recommendation_products.setdefault(row_val['ncodpers'],[]).append(product_cols[row_val['product']])
    if len(used_recommendation_products) % 100000 == 0:
        print(len(used_recommendation_products))

In [ ]:
#test_ids are the customer codes for the testing data.
test_ids = np.array(pd.read_csv(r'E:\Santander Recommendation\test_ver2.csv',usecols=['ncodpers'])['ncodpers'])
target_cols_all = np.array(product_cols)
final_preds = []
#iterate through our model's predictions (pred) and add the 7 most recommended products that the customer does not have.
for idx,predicted in enumerate(preds):
    ids = test_ids[idx]
    top_product = target_cols_all[predicted]
    used_products = used_recommendation_products.get(ids,[])
    new_top_product = []
    for product in top_product:
        if product not in used_products:
            new_top_product.append(product)
        if len(new_top_product) == 7:
            break
    final_preds.append(' '.join(new_top_product))
    if len(final_preds) % 100000 == 0:
        print(len(final_preds))

In [ ]:
submission = pd.DataFrame({'ncodpers':test_ids,'added_products':final_preds})
submission.to_csv('submission.csv',index=False)

In [ ]:
submission.shape

In [ ]:
submission.head()